In [ ]:
from keras.utils import np_utils
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import matplotlib.pyplot as plt
import cv2
import os
import json
import csv
import collections
import pickle


<span style='background-color:#fff5b1'>data_dir는 사용자에 맞게 변경 필요</span>
------------------------------------------------

In [ ]:
data_dir = 'C:/Users/kungm/Desktop/ETRI/example/'

Data preprocessing
=============

<span style='background-color:#fff5b1'>file_path, wav_path, annotation_root는 사용자에 맞게 변경 필요</span>
-----------------------------------------------------------------------------------------------------------
모든 session.csv 파일 불러와서 id,emotion,label,wav_path 정보만 담아서 합치기
train_set : test set = 8 :2

In [ ]:
def column_extraction(filepath):
    df = pd.read_csv(filepath)
    col = df.columns

    eval_col = [index for index in col if 'Eval' in index]

    eval_df = df['Segment ID']
    for index in eval_col:
        _df = df[index]
        eval_df = pd.concat([eval_df, _df], axis=1)

    eval_df.drop([0], axis=0, inplace=True)
    #19파일은 .csv파일 에 f/m가 들어가며 그거에 따라서 화자/청자가 정해짐. 그거 구분 위함
    for i in eval_df.index:
        if (filepath[-9:-8] != eval_df['Segment ID'][i][-4:-3]):
            eval_df.drop(i, axis=0, inplace=True)
        
    eval_df.reset_index(drop=True, inplace=True)

    indices = np.arange(len(eval_df))
    np.random.shuffle(indices)

    return eval_df, indices


def process_emotion(dataframe, dictionary, idx_array):
    for i in idx_array:
        total_eval = dataframe.iloc[i, 1]
        eval_sum = dataframe.iloc[i, 2:].to_list()
        eval_dict = dict(collections.Counter(eval_sum))

        for key, value in eval_dict.items():
            if key in total_eval:
                file_path = f"C:/Users/kungm/Desktop/ETRI/KEMDy19/wav/Session{dataframe.iloc[i, 0][4:6]}/{dataframe.iloc[i, 0][:-5]}/{dataframe.iloc[i, 0]}.txt"
                wav_path = f"C:/Users/kungm/Desktop/ETRI/KEMDy19/wav/Session{dataframe.iloc[i, 0][4:6]}/{dataframe.iloc[i, 0][:-5]}/{dataframe.iloc[i, 0]}.wav"
                if not os.path.exists(file_path):
                    continue
                
                with open(file_path, 'r', encoding='utf-8') as file:
                    file_content = file.read()
                    file_content = file_content.replace('c/', '').replace('n/', '').replace('N/', '').replace('u/', '').replace('l/', '').replace('b/', '').replace('s/', '').replace('o/', '').replace('*', '').replace('+', '').replace('/', '').replace('(', '').replace(')', '')
                if (value >= 6):
                    if (key == 'fear'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=0, Path=wav_path)
                    elif (key == 'surprise'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=1, Path=wav_path)
                    elif (key == 'angry'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=2, Path=wav_path)
                    elif (key == 'sad'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=3, Path=wav_path)
                    elif (key == 'neutral'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=4, Path=wav_path)
                    elif (key == 'happy'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=5, Path=wav_path)
                    elif (key == 'disgust'):
                        dictionary[dataframe.iloc[i, 0]] = dict(Emotion=key, Text=file_content[:-1], Label=6, Path=wav_path)


def save_json(dictionary, file_name):
    json_data = json.dumps(dictionary, ensure_ascii=False)
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(json_data)
        
def json_to_csv(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        
        # Write header
        writer.writerow(['ID', 'Emotion', 'Text', 'Label', 'Path'])
        
        # Write data
        for key, value in data.items():
            row = [key, value['Emotion'], value['Text'], value['Label'], value['Path']]
            writer.writerow(row)
        

def main():
    train_final = dict()
    test_final = dict()


    annotation_root = "C:/Users/kungm/Desktop/ETRI/KEMDy19/annotation"
    csv_files = os.listdir(annotation_root)


    for csv_file in csv_files:
        csv_path = annotation_root + '/' + csv_file
        dataframe, idx_array = column_extraction(csv_path)
        
        train_indices = idx_array[:int(len(dataframe) * 0.8)]
        test_indices = idx_array[int(len(dataframe) * 0.8):]
        
        process_emotion(dataframe, train_final, train_indices)
        process_emotion(dataframe, test_final, test_indices)

    save_json(train_final, data_dir + 'train.json')
    save_json(test_final, data_dir + 'test.json')
    
    json_to_csv(data_dir + 'train.json', data_dir + 'train.csv')
    json_to_csv(data_dir + 'test.json', data_dir + 'val.csv')

    


if __name__ == "__main__":
    main()

.json 파일 tranpose 후 다시 저장
--------------

In [ ]:
train=[]
test=[]

with open(data_dir+"train.json", 'r',encoding='UTF8') as file:
    train = json.load(file)
    
with open(data_dir+"test.json", 'r',encoding='UTF8') as file:
    test = json.load(file)


In [ ]:
#오디오 전처리에 사용하기 편하게 파일정리

train_df = pd.DataFrame(train)
train_df=train_df.transpose()
train_df=train_df.reset_index()
train_df=train_df.rename(columns={'index':'ID'})
train_df=train_df.drop(columns={'Text'})


test_df = pd.DataFrame(test)
test_df=test_df.transpose()
test_df=test_df.reset_index()
test_df=test_df.rename(columns={'index':'ID'})
test_df=test_df.drop(columns={'Text'})


In [ ]:
train_audio_tranpose=train_df.to_dict()
test_audio_tranpose=test_df.to_dict()

def save_json(dictionary, file_name):
    json_data = json.dumps(dictionary, ensure_ascii=False)
    with open(file_name, 'w', encoding='UTF8') as f:
        f.write(json_data)
        
save_json(train_audio_tranpose,  data_dir + 'train_audio.json')
save_json(test_audio_tranpose,  data_dir + 'test_audio.json')

In [ ]:
#### json 파일 다시 불러와서 확인하는 부분 ####

train=[]
test=[]

with open(data_dir+'train_audio.json', 'r',encoding='UTF8') as file:
    train = json.load(file)
    
with open(data_dir+'test_audio.json', 'r',encoding='UTF8') as file:
    test = json.load(file)
    

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [ ]:
train_df

In [ ]:
test_df

Extraction feature
======================
<span style='background-color:#fff5b1'>custpath 사용자에 맞게 변경 필요,</span>
<span style='background-color:#fff5b1'>Wav 데이터 필요</span>
------------------------------------------------

 -Mel Spectrogram

In [ ]:
custpath = 'C:/Users/kungm/Desktop/ETRI/example/'#spectrogram 이미지 저장

def feature_extractor(row):

    name     = row[0]
    y, sr = librosa.load(row[3], sr=22050)
    try:
        
        audio = y
    
        #Spectrogram
        plt.axis('off') # no axis(선선)
        plt.axes([0., 0., 1., 1.])
        melspec  = librosa.feature.melspectrogram(y=audio)
        s_db     = librosa.power_to_db(melspec, ref=np.max) #소리 변환을 기록으로 인식할 수 있도록크기를 dB(데시벨)로 반전 크기를 전환합니다.
        librosa.display.specshow(s_db)
        savepath = os.path.join(custpath,name+'.png')
        plt.savefig(savepath, bbox_inches=None, pad_inches=0)
        plt.close()
        
    except:
        print('File cannot open:',name)
        return None
    return savepath

def resize_img(datafram):
    for i in tqdm(range(len(datafram))):
        file_path = datafram[i]
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224),interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(file_path, img)

In [ ]:
imgpaths_train = []
imgpaths_test = []

idx = 0


for row in tqdm(train_df.values, leave=True):
    Feature_Set  = feature_extractor(row)
    imgpaths_train.append(Feature_Set)
    

for row in tqdm(test_df.values, leave=True):
    Feature_Set  = feature_extractor(row)
    imgpaths_test.append(Feature_Set)

#ResNet50 input 형식에 맞게 변경
resize_img(imgpaths_train)
resize_img(imgpaths_train)

Data generation
==============================

In [ ]:
data_train={}
data_train['ID']=[]
data_train['path']=[]
data_train['emotion']=[]

data_train_feature= [imgpaths_train[i] for i in range(len(imgpaths_train))]

for i in range(len(train_df)):
    data_train['ID'].append(train_df['ID'][i])
    data_train['path'].append(data_train_feature[i])
    data_train['emotion'].append(train_df['Label'][i])
    
data_test={}
data_test['ID']=[]
data_test['path']=[]
data_test['emotion']=[]

data_test_feature= [imgpaths_test[i] for i in range(len(imgpaths_test))]


for i in range(len(test_df)):
    data_test['ID'].append(test_df['ID'][i])
    data_test['path'].append(data_test_feature[i])
    data_test['emotion'].append(test_df['Label'][i])

In [ ]:

pickle.dump( data_train, open( data_dir + 'data_train.pickle', 'wb' ) )
pickle.dump( data_test, open( data_dir + 'data_test.pickle', 'wb' ) )

In [ ]:
#### data 파일 다시 불러와서 확인하는 부분 ####

data_train={}
data_train['ID']=[]
data_train['path']=[]
data_train['emotion']=[]

data_test={}
data_test['ID']=[]
data_test['path']=[]
data_test['emotion']=[]

with open( data_dir + 'data_train.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_train = pickle.load(f)
    


with open( data_dir + 'data_test.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_test = pickle.load(f)

        
ds_train = pd.DataFrame.from_dict(data_train)
ds_test = pd.DataFrame.from_dict(data_test)

In [ ]:
ds_train

In [ ]:
ds_test

Spectrogram이 없는 사용자도 모델을 학습시킬 수 있는 data 생성
==============================

In [ ]:
train={}
train['ID']=[]
train['path']=[]
train['emotion']=[]

for i in tqdm(range(len(data_train['path']))):
    seg_id= data_train['ID'][i]
    train['ID'].append(seg_id)
    img = cv2.imread(data_train['path'][i])
    train['path'].append(img)
    emotion = data_train['emotion'][i]
    train['emotion'].append(emotion)
    
    
test={}
test['ID']=[]
test['path']=[]
test['emotion']=[]

for i in tqdm(range(len(data_test['path']))):
    seg_id= data_test['ID'][i]
    test['ID'].append(seg_id)
    img = cv2.imread(data_test['path'][i])
    test['path'].append(img)
    emotion = data_test['emotion'][i]
    test['emotion'].append(emotion)
        

In [ ]:

pickle.dump( train, open( data_dir + 'train_final.pickle', 'wb' ) )
pickle.dump( test, open( data_dir + 'test_final.pickle', 'wb' ) )

In [ ]:
train={}
train['ID']=[]
train['path']=[]
train['emotion']=[]
    
test={}
test['ID']=[]
test['path']=[]
test['emotion']=[]

with open( data_dir + 'train_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    train = pickle.load(f)
    


with open( data_dir + 'test_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    test = pickle.load(f)

        
ds_train = pd.DataFrame.from_dict(train)
ds_test = pd.DataFrame.from_dict(test)

In [ ]:
ds_test

In [ ]:
ds_train